Conexão com o Azure Data Lake

In [1]:
#importando as bibliotecas
from azure.storage.blob import BlobServiceClient
from io import BytesIO
import pandas as pd

In [3]:

# Chaves de acesso e URL do serviço
account_key = "SuaChaveDeAcesso"
account_name = "NomeDaConta"
container_name = "NomeDoContainer"

# Cria o cliente de serviço
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)


Leitura dos arquivos csv no container

In [4]:

def ler_arquivo_csv(account_key, account_name, container_name, blob_name, delimitador=';'):
    # Cria o cliente de serviço
    blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

    # Obtém referência para o blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    # Lê o conteúdo do blob (arquivo CSV) em um DataFrame
    blob_content = blob_client.download_blob().readall()
    # Especifique o delimitador usando o parâmetro sep
    df = pd.read_csv(BytesIO(blob_content), sep=delimitador)

    return df

# Exemplo de uso
df_stores = ler_arquivo_csv(account_key, account_name, container_name, "Excel/Stores.csv")
df_product = ler_arquivo_csv(account_key, account_name, container_name, "Excel/Product.csv")


In [5]:
df_stores

,StoreKey,GeographyKey,StoreType,StoreName,Status,CloseReason,EmployeeCount,SellingAreaSize
0,1,693,Store,Contoso Seattle No.1 Store,On,NaN,17.0,462
1,2,693,Store,Contoso Seattle No.2 Store,On,NaN,25.0,700
2,3,856,Store,Contoso Kennewick Store,On,NaN,26.0,680
3,4,424,Store,Contoso Bellevue Store,On,NaN,19.0,455
4,5,677,Store,Contoso Redmond Store,On,NaN,33.0,560
...,...,...,...,...,...,...,...,...
301,306,586,Online,Contoso Europe Online Store,On,NaN,7.0,1000
302,307,710,Online,Contoso Asia Online Store,On,NaN,8.0,93800
303,308,693,Reseller,Contoso North America Reseller,On,NaN,15.0,450
304,309,529,Reseller,Contoso Europe Reseller,On,NaN,12.0,551


In [6]:
df_product

,ProductName,ProductDescription,Manufacturer,BrandName,ClassName,UnitCost,UnitPrice,ProductKey,ProductSubcategoryKey
0,Contoso Wireless Laser Mouse E50 Grey,Advanced 2.4 GHz cordless technology makes fre...,"Contoso, Ltd",Contoso,Economy,"10,69","20,96",873,22
1,Contoso Optical Wheel OEM PS/2 Mouse E60 Grey,"PS/2 mouse, 6 feet mouse cable","Contoso, Ltd",Contoso,Economy,"6,63",13,879,22
2,Contoso Optical Wheel OEM PS/2 Mouse E60 Black,"PS/2 mouse, 6 feet mouse cable","Contoso, Ltd",Contoso,Economy,"6,63",13,880,22
3,Contoso Optical Wheel OEM PS/2 Mouse E60 White,"PS/2 mouse, 6 feet mouse cable","Contoso, Ltd",Contoso,Economy,"6,63",13,881,22
4,Contoso Optical Wheel OEM PS/2 Mouse E60 Silver,"PS/2 mouse, 6 feet mouse cable","Contoso, Ltd",Contoso,Economy,"6,63",13,882,22
...,...,...,...,...,...,...,...,...,...
1685,SV DVD 12-Inch Player Portable M400 Black,"6-hour rechargeable battery, 180-degree swivel...",Southridge Video,Southridge Video,Regular,"82,77","179,99",1602,35
1686,SV DVD 9-Inch Player Portable M300 Silver,"6-hour rechargeable battery, 180-degree swivel...",Southridge Video,Southridge Video,Regular,"73,57","159,99",1606,35
1687,SV DVD 12-Inch Player Portable M400 Silver,"6-hour rechargeable battery, 180-degree swivel...",Southridge Video,Southridge Video,Regular,"82,77","179,99",1607,35
1688,SV DVD 9-Inch Player Portable M300 White,"6-hour rechargeable battery, 180-degree swivel...",Southridge Video,Southridge Video,Regular,"73,57","159,99",1611,35


Lista todos os blobs do azure storage

In [7]:
# Obtém referência para o contêiner
container_client = blob_service_client.get_container_client(container_name)

# Lista os blobs no contêiner
blobs = container_client.list_blobs()

# Itera sobre os blobs e faz alguma coisa (por exemplo, imprime o nome do blob)
for blob in blobs:
    print(f"Nome do Blob: {blob.name}")

# Lê o conteúdo do blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)
conteudo_blob = blob_client.download_blob()


Nome do Blob: Carros.csv
Nome do Blob: Excel
Nome do Blob: Excel/Calendar.csv
Nome do Blob: Excel/Channel.csv
Nome do Blob: Excel/Geography.csv
Nome do Blob: Excel/Product.csv
Nome do Blob: Excel/ProductCategory.csv
Nome do Blob: Excel/ProductSubcategory.csv
Nome do Blob: Excel/Sales.csv
Nome do Blob: Excel/Stores.csv
Nome do Blob: raw


Conexão com o Azure SQL

In [8]:
#importando as bibliotecas
from sqlalchemy import create_engine
import pyodbc

In [9]:
def criar_conexao_azure_sql(server_name, database_name, username, password):
    # Define a string de conexão
    connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_name};DATABASE={database_name};UID={username};PWD={password};"

    # Cria a conexão
    conexao = pyodbc.connect(connection_string)

    return conexao

In [10]:
# Configurações de conexão
server_name = "seu_servidor.database.windows.net"
database_name = "seu_banco_de_dados"
username = "seu_usuario"
password = "sua_senha"


In [11]:

# Cria a string de conexão
connection_string = f"mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server"

# Cria uma engine de conexão usando SQLAlchemy
engine = create_engine(connection_string, echo=True)

# Inserir dados na tabela stg_stores
df_stores.to_sql(name="stg_stores", con=engine, if_exists="replace", index=False)

# Inserir dados na tabela stg_Product
df_product.to_sql(name="stg_product", con=engine, if_exists="replace", index=False)

# Fechar a conexão
engine.dispose()


2024-01-02 10:23:04,158 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-01-02 10:23:04,159 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-02 10:23:04,292 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-01-02 10:23:04,292 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ()
2024-01-02 10:23:04,556 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-01-02 10:23:04,557 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ()
2024-01-02 10:23:04,829 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-02 10:23:04,831 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ('BASE TABLE', 'stg_stores', 'dbo')
2